In [ ]:
!pip install influxdb_client
#3st Stage - Sessions connector
# sessions
# table name: Fact_User_session

import pandas
from pandas import DataFrame
import os
import influxdb_client
from influxdb_client.client import query_api
import datetime

my_org = os.environ.get('InfluxDB_org')
client = influxdb_client.InfluxDBClient(url = os.environ.get('InfluxDB_URL'), token = os.environ.get('InfluxDB_token'), org = my_org)

query_api = client.query_api()


In [ ]:
# (0) Get values to dataframe
query = 'from(bucket: "hybrid-dast")\
|> range(start: v.timeRangeStart, stop: v.timeRangeStop)\
|> filter(fn: (r) => r["_measurement"] == "pings" and r.channel == "57ea31912bb6581000092e89")\
|> aggregateWindow(every: 1h, fn: count, createEmpty: true)'

data = client.query_api().query_data_frame(org = my_org, query = query)

In [ ]:
data

,result,table,_start,_stop,_time,_value,_field,_measurement,channel
0,_result,0,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:00:00.060000+00:00,5e5c100aaa3b86001eb0ae80,uid,pings,59f9c0dfbc95750d006b98f5
1,_result,0,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:00:00.228000+00:00,5f92f16d5d817400179697c1,uid,pings,59f9c0dfbc95750d006b98f5
2,_result,0,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:00:00.453000+00:00,5e3d8abe41eb00001e38eb16,uid,pings,59f9c0dfbc95750d006b98f5
3,_result,0,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:00:00.591000+00:00,5fb928b73c1290001707a94d,uid,pings,59f9c0dfbc95750d006b98f5
4,_result,0,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:00:00.609000+00:00,5e41bd10980315001e167ee3,uid,pings,59f9c0dfbc95750d006b98f5
...,...,...,...,...,...,...,...,...,...
536672,_result,1,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:57.461000+00:00,5f9305fa16de4a00172ca837,uid,pings,57ea31912bb6581000092e89
536673,_result,1,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:57.496000+00:00,5ff782a2450cbb0017a24261,uid,pings,57ea31912bb6581000092e89
536674,_result,1,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:57.507000+00:00,5e3ebc0d39a96c001e50e358,uid,pings,57ea31912bb6581000092e89
536675,_result,1,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:57.520000+00:00,5e3d539288b36b001e882a15,uid,pings,57ea31912bb6581000092e89


In [ ]:
# drop unnecessary tables
data.drop(columns=['result', 'table', '_measurement'], inplace=True)

In [ ]:
# (2) Calculate time difference between dates per row to get star and end session 

#get time difference (seconds) for each related datetime by user and channel
df = data.sort_values(['_value', '_time'])
df['Diff'] = df.groupby(['_value', 'channel'])['_time'].diff().dt.seconds


# get consecutive groups of any values and multiply total number of consecutive values by the 'number' to get total seconds for group
df['times_value'] = df.Diff.groupby((df.Diff != df.Diff.shift()).cumsum()).transform('size') * df.Diff

In [ ]:
# assign total duration between start and end session to row sessions
import numpy as np

#get value of the next row in"times_value" if Diff value is greater than 16sec and next value in row is not consecutive
df['Post_Time'] = df.loc[((df['Diff'].shift(+1) >= 16) | (np.isnan((df['Diff'].shift(+1))))  ) &  (df['Diff'].shift(0) <= 15), 'times_value']
df['Post_Time'] = df['Post_Time'].shift(-1)

In [ ]:
# (3) Session start

# conditions for session start: get rows starts/with values greater than 15 sec (>=16) or null (start of new user), get as a start value
x = df[df['Diff']  >= 16]
y = df[df['Diff'].isnull()]

#get dataframe with rows for each session with 'start_session' value
new_start = pandas.concat([x, y])
new_start

,_start,_stop,_time,_value,_field,channel,Diff,times_value,Post_Time
458060,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:57:56.485000+00:00,5e3d2735ffc977001ea26e45,uid,57ea31912bb6581000092e89,30.0,30.0,45.0
264971,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:01:19.705000+00:00,5e3d2797ffc977001ea26efd,uid,57ea31912bb6581000092e89,33.0,33.0,240.0
279874,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:05:50.859000+00:00,5e3d2797ffc977001ea26efd,uid,57ea31912bb6581000092e89,30.0,30.0,15.0
281861,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:06:36.044000+00:00,5e3d2797ffc977001ea26efd,uid,57ea31912bb6581000092e89,30.0,30.0,15.0
284591,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:07:36.300000+00:00,5e3d2797ffc977001ea26efd,uid,57ea31912bb6581000092e89,45.0,45.0,75.0
...,...,...,...,...,...,...,...,...,...
534991,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:22.401000+00:00,5ff8a2de803fdd0017ec3cac,uid,57ea31912bb6581000092e89,NaN,NaN,NaN
536333,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:50.291000+00:00,5ff8a2fa803fdd0017ec3f32,uid,57ea31912bb6581000092e89,NaN,NaN,NaN
536389,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:51.432000+00:00,5ff8a2fb06a55500174e6358,uid,57ea31912bb6581000092e89,NaN,NaN,NaN
131626,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 20:18:08.205000+00:00,5ff8be00803fdd0017ef21ae,uid,59f9c0dfbc95750d006b98f5,NaN,NaN,NaN


In [ ]:
# check result [dataframe with start_session]
new_start.to_csv('start.csv')

In [ ]:
# (4) End session
import datetime

#prepare: convert null values in column to 0 to compute sum
new_start['Post_Time'] = new_start['Post_Time'].fillna(0)
new_start

# sum 'times value'/duration with 'start date' for each row to get 'end session'
new_start['end_session'] = new_start.apply(lambda row: row._time + datetime.timedelta(seconds = row.Post_Time), axis=1)


In [ ]:
# check result [dataframe with end_session]
new_start

,_start,_stop,_time,_value,_field,channel,Diff,times_value,Post_Time,end_session
458060,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:57:56.485000+00:00,5e3d2735ffc977001ea26e45,uid,57ea31912bb6581000092e89,30.0,30.0,45.0,2021-01-08 17:58:41.485000+00:00
264971,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:01:19.705000+00:00,5e3d2797ffc977001ea26efd,uid,57ea31912bb6581000092e89,33.0,33.0,240.0,2021-01-08 17:05:19.705000+00:00
279874,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:05:50.859000+00:00,5e3d2797ffc977001ea26efd,uid,57ea31912bb6581000092e89,30.0,30.0,15.0,2021-01-08 17:06:05.859000+00:00
281861,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:06:36.044000+00:00,5e3d2797ffc977001ea26efd,uid,57ea31912bb6581000092e89,30.0,30.0,15.0,2021-01-08 17:06:51.044000+00:00
284591,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 17:07:36.300000+00:00,5e3d2797ffc977001ea26efd,uid,57ea31912bb6581000092e89,45.0,45.0,75.0,2021-01-08 17:08:51.300000+00:00
...,...,...,...,...,...,...,...,...,...,...
534991,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:22.401000+00:00,5ff8a2de803fdd0017ec3cac,uid,57ea31912bb6581000092e89,NaN,NaN,0.0,2021-01-08 18:22:22.401000+00:00
536333,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:50.291000+00:00,5ff8a2fa803fdd0017ec3f32,uid,57ea31912bb6581000092e89,NaN,NaN,0.0,2021-01-08 18:22:50.291000+00:00
536389,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 18:22:51.432000+00:00,5ff8a2fb06a55500174e6358,uid,57ea31912bb6581000092e89,NaN,NaN,0.0,2021-01-08 18:22:51.432000+00:00
131626,2021-01-08 17:00:00+00:00,2021-01-09 00:00:00+00:00,2021-01-08 20:18:08.205000+00:00,5ff8be00803fdd0017ef21ae,uid,59f9c0dfbc95750d006b98f5,NaN,NaN,0.0,2021-01-08 20:18:08.205000+00:00


In [ ]:
# (5) Final formatting

# drop unnecessary columns
new_start.drop(columns=['_start', '_stop', '_field', 'Diff', 'times_value'], inplace=True)

# rename dataframe
new_start.columns = ['Session_start', 'DeviceUser_id', 'Channel_id', 'Duration', 'Session_end']


# re-order
new_start = new_start[['DeviceUser_id', 'Channel_id', 'Session_start', 'Session_end', 'Duration']]

new_start

,DeviceUser_id,Channel_id,Session_start,Session_end,Duration
458060,5e3d2735ffc977001ea26e45,57ea31912bb6581000092e89,2021-01-08 17:57:56.485000+00:00,2021-01-08 17:58:41.485000+00:00,45.0
264971,5e3d2797ffc977001ea26efd,57ea31912bb6581000092e89,2021-01-08 17:01:19.705000+00:00,2021-01-08 17:05:19.705000+00:00,240.0
279874,5e3d2797ffc977001ea26efd,57ea31912bb6581000092e89,2021-01-08 17:05:50.859000+00:00,2021-01-08 17:06:05.859000+00:00,15.0
281861,5e3d2797ffc977001ea26efd,57ea31912bb6581000092e89,2021-01-08 17:06:36.044000+00:00,2021-01-08 17:06:51.044000+00:00,15.0
284591,5e3d2797ffc977001ea26efd,57ea31912bb6581000092e89,2021-01-08 17:07:36.300000+00:00,2021-01-08 17:08:51.300000+00:00,75.0
...,...,...,...,...,...
534991,5ff8a2de803fdd0017ec3cac,57ea31912bb6581000092e89,2021-01-08 18:22:22.401000+00:00,2021-01-08 18:22:22.401000+00:00,0.0
536333,5ff8a2fa803fdd0017ec3f32,57ea31912bb6581000092e89,2021-01-08 18:22:50.291000+00:00,2021-01-08 18:22:50.291000+00:00,0.0
536389,5ff8a2fb06a55500174e6358,57ea31912bb6581000092e89,2021-01-08 18:22:51.432000+00:00,2021-01-08 18:22:51.432000+00:00,0.0
131626,5ff8be00803fdd0017ef21ae,59f9c0dfbc95750d006b98f5,2021-01-08 20:18:08.205000+00:00,2021-01-08 20:18:08.205000+00:00,0.0


In [ ]:
new_start.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42701 entries, 458060 to 203824
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype                    
---  ------         --------------  -----                    
 0   DeviceUser_id  42701 non-null  object                   
 1   Channel_id     42701 non-null  object                   
 2   Session_start  42701 non-null  datetime64[ns, tzlocal()]
 3   Session_end    42701 non-null  datetime64[ns, tzlocal()]
 4   Duration       42701 non-null  float64                  
dtypes: datetime64[ns, tzlocal()](2), float64(1), object(2)
memory usage: 2.0+ MB


In [ ]:
new_start.to_csv('fact_user_session.csv')

In [ ]:
#!pip install snowflake-connector-python
import pandas as pd
import snowflake.connector

/usr/local/lib/python3.6/dist-packages/snowflake/connector/options.py:78: UserWarning: You have an incompatible version of 'pyarrow' installed (0.14.1), please install a version that adheres to: 'pyarrow<0.18.0,>=0.17.0; extra == "pandas"'
  warn_incompatible_dep('pyarrow', _installed_pyarrow_version.version, _expected_pyarrow_version)


In [ ]:
conn = snowflake.connector.connect(
                user="os.environ.get('SNOWFLAKE_USER')",
                password="os.environ.get('SNOWFLAKE_PASS')",
                account="os.environ.get('SNOWFLAKE_ACC')",
                database="os.environ.get('SNOWFLAKE_DB')",
                schema="os.environ.get('SNOWFLAKE_SCHEMA')")

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute('USE WAREHOUSE AFTERSPOT_WH')

In [ ]:
cur.execute("put file://{} @{} auto_compress=true".format('fact_user_session.csv', 'TEST'))

In [ ]:
cur.execute("COPY INTO FACT_USER_SESSION FROM @test/fact_user_session.csv.gz FILE_FORMAT=(TYPE=csv field_delimiter=',' skip_header=1)")

In [ ]:
cur.execute("rm @test/fact_user_session.csv.gz;")